In [ ]:
import os
import arcpy

In [4]:
# ========= CONFIG =========
region    = "KHMAO"
gdb_path  = r"C:\Users\User\Documents\ЦГД\KHMAO_RiskProfile_27-10-2025\KHMAO_RiskProfile\Default.gdb"
RISKS_FDS = "risk"   # feature dataset that holds risk layers
DRY_RUN   = False     # True -> don't modify, just count & print

fields_to_fix = ["hExpertTot", "hExpertLive", "hNormLive", "hNormTot"]
old_value = "умерено опасно"
new_value = "умеренно опасно"

# ========= RUN =========
arcpy.env.workspace = os.path.join(gdb_path, RISKS_FDS)
fcs = arcpy.ListFeatureClasses() or []

total_rows_scanned = 0
total_rows_changed = 0
total_fc_changed   = 0

print(f"[ValueFix] Dataset: {RISKS_FDS} | FC count: {len(fcs)}")
for fc in fcs:
    fc_path = os.path.join(gdb_path, RISKS_FDS, fc)

    # skip if none of the target fields exist
    existing = {f.name for f in arcpy.ListFields(fc_path)}
    targets  = [f for f in fields_to_fix if f in existing]
    if not targets:
        continue

    changed_rows = 0
    scanned_rows = 0

    # use UpdateCursor if not dry-run, else SearchCursor
    Cursor = arcpy.da.UpdateCursor if not DRY_RUN else arcpy.da.SearchCursor
    with Cursor(fc_path, targets) as cur:
        for row in cur:
            scanned_rows += 1
            if DRY_RUN:
                # just count potential changes
                if any(isinstance(v, str) and (old_value in v) for v in row):
                    changed_rows += 1
                continue

            # live replace
            row_list = list(row)
            touched = False
            for i, v in enumerate(row_list):
                if isinstance(v, str) and (old_value in v):
                    row_list[i] = v.replace(old_value, new_value)
                    touched = True
            if touched:
                cur.updateRow(row_list)
                changed_rows += 1

    if changed_rows:
        total_fc_changed += 1
        print(f"  [{fc}] fields {targets} -> updated rows: {changed_rows} (scanned: {scanned_rows})")
    total_rows_scanned += scanned_rows
    total_rows_changed += changed_rows

print(f"[ValueFix] DONE. DRY_RUN={DRY_RUN}. FCs changed: {total_fc_changed}, "
      f"rows changed: {total_rows_changed}, rows scanned: {total_rows_scanned}")


[ValueFix] Dataset: risk | FC count: 11
[ValueFix] DONE. DRY_RUN=False. FCs changed: 0, rows changed: 0, rows scanned: 242
